# Przetwarzanie Grafiki i Muzyki - laboratorium nr 7

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Binaryzacja.

Binaryzacja polega na zamianie obrazów monochromatycznych na obrazy binarne. Przekształcenie to jest prawie zawsze wykorzystywane w analizie obrazów, gdyż wiele operacji może być przeprowadzonych wyłącznie na obiektach binarnych.

## Zadanie 1 (1 punkt)

Proszę zaimplementować binaryzację z zadanym poziomem ( a ).

$$
J(i,j) = \left\{ \begin{array}{ll}
0 & \textrm{gdy } J(i,j)<a \\
255 & \textrm{gdy } J(i,j) \geq a 
\end{array} \right.
$$

  * Proszę zastosować do zdjęcia "LENA_512.jpg". 
  * Proszę wykonać na Luminancji (składowa Y z formatu YCbCr). 

In [2]:
def binarization(img):
    a = 127
    new_img = ((img >= a).astype(np.uint8) * 255)
    cv2.imshow('before', img)
    cv2.imshow('after', new_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

img = cv2.imread('./img/LENA_512.jpg')
ycbcr_img = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
y_channel = ycbcr_img[:, :, 0]
binarization(y_channel)

## Zadanie 2 (1 punkt) 

Proszę zaimplementować binaryzację z zadanym przedziałem ($[a,b]$):

$$
J(i,j) = \left\{ \begin{array}{ll}
255 & \textrm{gdy } J(i,j) \in \left[a,b\right] \\
0 & \textrm{w przeciwnym przypadku} 
\end{array} \right.
$$

Zastosuj tę binaryzację do obrazu "LENA_512.jpg".

In [3]:
def binarization(img):
    a = 100
    b = 127
    mask = ((img >= a) & (img <= b)).astype(np.uint8)
    new_img = mask * 255
    cv2.imshow('before', img)
    cv2.imshow('after', new_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

img = cv2.imread('./img/LENA_512.jpg')
ycbcr_img = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
y_channel = ycbcr_img[:, :, 0]
binarization(y_channel)

# Zadanie 3 (2 punkty)

Prosze użyć algorytm binaryzacji Otsu do zdjęcia "gazeta.jpg". 

![alt](https://raw.githubusercontent.com/przem85/PGiM/master/img/gazeta.jpg)

Proszę wykonać na Luminancji (składowa Y z formatu YCbCr).

In [16]:
# cv2.THRESHOLD(img, ..., FLAG)
# FLAG = cv2.THRESH_BINARY + cv2.THRESH_OTSU

img = cv2.imread('./img/gazeta.jpg')
ycbcr_img = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
y_channel = ycbcr_img[:, :, 0]
a = 100
ret, thresh1 = cv2.threshold(y_channel, a, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
cv2.imshow('Otsu Threshold', thresh1)
cv2.waitKey(0)
cv2.destroyAllWindows()

105.0


# Zadanie 4 (3 punkty)

Proszę zaimplementować algorytm binaryzacji Bernsena. Metoda Bernsena to algorytm wyznaczania progu lokalnego w oparciu o otoczenie piksela (zwykle kwadratowe okno o nieparzystej szerokości). 
Próg ustawiany jest na wartość średnią najjaśniejszego i najciemniejszego piksela.

  * Proszę użyć algorytm binaryzacji Bernsena do zdjęcia "gazeta.jpg".
  * Proszę wykonać na Luminancji (składowa Y z formatu YCbCr).

Algorytm Brensena:
1. Przekształcamy oryginalny obrazek do skali szarości. Możemy teraz myśleć o obrazku, jak o pewnej funkcji $I\left(x,y\right): \mathbf{N}^2 \rightarrow \mathbf{N}$ (zakładając, że poruszamy się po świecie liczb naturalnych).
2. Wokół każdego piksela bierzemy kwadrat o nieparzystej długości $a>0$ i wyliczamy wartość minimalną oraz maksymalną z intensywności pikseli w każdym takim kwadracie. Formalnie, dla każdego punktu $\left(x,y\right)\in\mathbf{N}^2$ i ustalonego $a>0$ wyliczamy:

\begin{align}
  v_{min}\left(x,y;a\right) &= \min{ \left( I\left(i,j\right): x-a \leq i \leq x+a, \quad y-a \leq j \leq y+a \right) } \\
  v_{max}\left(x,y;a\right) &= \max{ \left( I\left(i,j\right): x-a \leq i \leq x+a, \quad y-a \leq j \leq y+a \right) }
\end{align}
3. Dla każdego kwadratu wyliczamy próg $T$ jako $T = \frac{v_{min}\left(x,y;a\right) + v_{max}\left(x,y;a\right)}{2}$.

In [21]:
# 1. Konwersja na Grayscale
# 2. Dla każdej intensywności piksela bierzemy kwadrat liczb i wyliczamy wartość minimalną i maksymalną 
# 3. T = (maks + min) / 2

import cv2
import numpy as np

def bernsen_thresholding(image, window_size):
    rows, cols = image.shape
    binary_image = np.zeros_like(image, dtype=np.uint8)

    for i in range(rows):
        for j in range(cols):
            row_start = max(0, i - window_size // 2)
            row_end = min(rows, i + window_size // 2 + 1)
            col_start = max(0, j - window_size // 2)
            col_end = min(cols, j + window_size // 2 + 1)

            min_val = np.min(image[row_start:row_end, col_start:col_end])
            max_val = np.max(image[row_start:row_end, col_start:col_end])
            threshold = (min_val + max_val) // 2

            if image[i, j] > threshold:
                binary_image[i, j] = 255
            else:
                binary_image[i, j] = 0

    return binary_image


image = cv2.imread("./img/gazeta.jpg")
image_YCrCb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
y_channel = image_YCrCb[:,:,0]
window_size = 15
binary_image = bernsen_thresholding(y_channel, window_size)

cv2.imshow("Original Image", image)
cv2.imshow("Bernsen Thresholding", binary_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


[[166  74 167 ... 222 220 223]
 [161  82 179 ... 246 253 241]
 [160  74 173 ... 241 244 246]
 ...
 [160  21  42 ... 148 148 147]
 [161  20  41 ... 148 148 147]
 [159  19  39 ... 146 146 146]]


C:\Users\HP\AppData\Local\Temp\ipykernel_5720\1068198571.py:22: RuntimeWarning: overflow encountered in ubyte_scalars
  threshold = (min_val + max_val) // 2


# Zadanie 5 (3 punkty)
Progowanie mieszane przebiega podobnie, jak progowanie lokalne (metoda Bernsena), z tym, że jeśli średnia lub mediana lokalna dla danego piksela odbiega o więcej niż ustalony próg (program pobiera tą wartość jako parametr) od wartości globalnej (wyznaczona za pomocą Otsu), piksel progowany jest wartością globalną.

  * Prosze użyć algorytm progowania mieszanego do zdjęcia "gazeta.jpg".
  * Proszę wykonać na Luminancji (składowa Y z formatu YCbCr).

  Algorytm progowania mieszanego:
  1. Wyznaczamy wartość progu globalnego przy pomocy binaryzacji Otsu. Oznaczmy ten próg jako $P$.
  2. Podobnie, jak w algorytmie Brensena, przekształcamy oryginalny obrazek do skali szarości.
  3.  Wokół każdego piksela bierzemy kwadrat o nieparzystej długości $a>0$ i wyliczamy wartość minimalną oraz maksymalną z intensywności pikseli w każdym takim kwadracie. Formalnie, dla każdego punktu $\left(x,y\right)\in\mathbf{N}^2$ i ustalonego $a>0$ wyliczamy:

\begin{align}
  v_{min}\left(x,y;a\right) &= \min{ \left( I\left(i,j\right): x-a \leq i \leq x+a, \quad y-a \leq j \leq y+a \right) } \\
  v_{max}\left(x,y;a\right) &= \max{ \left( I\left(i,j\right): x-a \leq i \leq x+a, \quad y-a \leq j \leq y+a \right) }
\end{align}

4. Dla każdego kwadratu wyliczamy próg $T$ jako $T = \frac{v_{min}\left(x,y;a\right) + v_{max}\left(x,y;a\right)}{2}$. Jeżeli jednak wartość tak wyliczonego progu $T$ różni się o więcej niż $\alpha > 0$ (parametr ten powinien być wczytywany jako argument funkcji) od progu globalnego $P$, to wtedy piksel progowany jest wartością globalną. Formalnie, jeśli $\left| T - \alpha \right| > P$, to wtedy jako próg lokalny przyjmujemy próg globalny $P$.

In [20]:
import cv2
import numpy as np

def mixed_thresholding(image, window_size, global_threshold, alpha):
    rows, cols = image.shape
    binary_image = np.zeros_like(image, dtype=np.uint8)

    for i in range(rows):
        for j in range(cols):
            row_start = max(0, i - window_size // 2)
            row_end = min(rows, i + window_size // 2 + 1)
            col_start = max(0, j - window_size // 2)
            col_end = min(cols, j + window_size // 2 + 1)

            local_min = np.min(image[row_start:row_end, col_start:col_end])
            local_max = np.max(image[row_start:row_end, col_start:col_end])
            local_threshold = (local_min + local_max) // 2

            if abs(local_threshold - alpha) > global_threshold:
                local_threshold = global_threshold
                
            if image[i, j] > local_threshold:
                binary_image[i, j] = 255
            else:
                binary_image[i, j] = 0

    return binary_image


image = cv2.imread("./img/gazeta.jpg")
image_YCrCb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
y_channel = image_YCrCb[:,:,0]
global_threshold, _ = cv2.threshold(y_channel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
window_size = 15  
alpha = 20
binary_image = mixed_thresholding(y_channel, window_size, global_threshold, alpha)

cv2.imshow("Original Image", image)
cv2.imshow("Mixed Thresholding", binary_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


C:\Users\HP\AppData\Local\Temp\ipykernel_5720\3579282717.py:17: RuntimeWarning: overflow encountered in ubyte_scalars
  local_threshold = (local_min + local_max) // 2
